<a href="https://colab.research.google.com/github/MaheelLakshan/Image_Segmentation_Thresholding_and_Region_Growing/blob/main/Question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import filters
